# Multiplicacion de matrices secuencial

In [21]:
import multiprocessing
import random
import time
from typing import List
import numpy as np

def generar_matriz(filas: int, columnas: int) -> List[List[int]]:
    return [[random.randint(1, 10) for _ in range(columnas)] for _ in range(filas)]

def multiplicar_matrices_secuencial(matriz_a: List[List[int]], matriz_b: List[List[int]]) -> List[List[int]]:
    filas_a = len(matriz_a)
    columnas_a = len(matriz_a[0])
    columnas_b = len(matriz_b[0])
    
    resultado: List[List[int]] = [[0 for _ in range(columnas_b)] for _ in range(filas_a)]
    
    for i in range(filas_a):
        for j in range(columnas_b):
            for k in range(columnas_a):
                resultado[i][j] += matriz_a[i][k] * matriz_b[k][j]
                
    return resultado

dimension: int = 250
matriz_a = generar_matriz(dimension, dimension)
matriz_b = generar_matriz(dimension, dimension)

# Multiplicación secuencial
inicio = time.time()
resultado_secuencial = multiplicar_matrices_secuencial(matriz_a, matriz_b)
fin = time.time()
tiempo_secuencial = fin - inicio
print(f"Tiempo de ejecución secuencial: {tiempo_secuencial:.4f} segundos")

Tiempo de ejecución secuencial: 1.4329 segundos


# Multiplicacion de matrices con multiprocesamiento

In [22]:
def multiplicar_fila(args: tuple) -> List[int]:
    fila_a, matriz_b, j = args
    resultado: List[int] = []
    for j in range(len(matriz_b[0])):
        suma = 0
        for k in range(len(matriz_b)):
            suma += fila_a[k] * matriz_b[k][j]
        resultado.append(suma)
    return resultado

def multiplicar_matrices_paralelo(matriz_a: List[List[int]], matriz_b: List[List[int]], num_procesos: int) -> List[List[int]]:
    
    with multiprocessing.Pool(processes=num_procesos) as pool:
        args: List[tuple] = [(fila_a, matriz_b, j) for j, fila_a in enumerate(matriz_a)]
        resultado: List[List[int]] = pool.map(multiplicar_fila, args)
    
    return resultado

dimension = 250
matriz_a = generar_matriz(dimension, dimension)
matriz_b = generar_matriz(dimension, dimension)
    
num_procesos = multiprocessing.cpu_count()
inicio = time.time()
resultado_paralelo = multiplicar_matrices_paralelo(matriz_a, matriz_b, num_procesos)
fin = time.time()
tiempo_paralelo = fin - inicio
print(f"Tiempo de ejecución paralelo ({num_procesos} cores): {tiempo_paralelo:.4f} segundos")    
print(f"Speedup: {tiempo_secuencial/tiempo_paralelo:.2f}x")

Tiempo de ejecución paralelo (8 cores): 0.3416 segundos
Speedup: 4.20x


# Multiplicación de matrices con numpy

In [23]:
dimension = 250
matriz_a = generar_matriz(dimension, dimension)
matriz_b = generar_matriz(dimension, dimension)

inicio = time.time()
resultado_numpy = np.dot(matriz_a, matriz_b)
fin = time.time()
tiempo_numpy = fin - inicio
print(f"Tiempo de ejecución con NumPy: {tiempo_numpy:.4f} segundos")
print(f"Speedup (NumPy): {tiempo_secuencial/tiempo_numpy:.2f}x")

Tiempo de ejecución con NumPy: 0.0291 segundos
Speedup (NumPy): 49.31x


# Multiplicación de matrices con torch CPU y CUDA

In [24]:
import torch

dimension = 250
matriz_a = generar_matriz(dimension, dimension)
matriz_b = generar_matriz(dimension, dimension)

a = torch.tensor(matriz_a, dtype=torch.float32)
b = torch.tensor(matriz_b, dtype=torch.float32)

#warmup 
_ = torch.matmul(a, b)


inicio = time.time()
resultado_torch_cpu = torch.matmul(a, b)
fin = time.time()
tiempo_torch_cpu = fin - inicio
print(f"Tiempo de ejecución con PyTorch (CPU): {tiempo_torch_cpu:.4f} segundos")
print(f"Speedup (PyTorch CPU): {tiempo_secuencial/tiempo_torch_cpu:.2f}x")

Tiempo de ejecución con PyTorch (CPU): 0.0003 segundos
Speedup (PyTorch CPU): 4202.77x


In [25]:

if not torch.cuda.is_available():
    print("No se encontraron GPUs disponibles")

else:    
    device = torch.device("cuda")

    dimension = 250
    matriz_a = generar_matriz(dimension, dimension)
    matriz_b = generar_matriz(dimension, dimension)

    a = torch.tensor(matriz_a, dtype=torch.float32)
    b = torch.tensor(matriz_b, dtype=torch.float32)

    a = a.to(device)
    b = b.to(device)

    #warmup 
    _ = torch.matmul(a, b)

    inicio = time.time()
    resultado_torch_cpu = torch.matmul(a, b)
    fin = time.time()
    tiempo_torch_cpu = fin - inicio
    print(f"Tiempo de ejecución con PyTorch (GPU): {tiempo_torch_cpu} segundos")
    print(f"Speedup (PyTorch GPU): {tiempo_secuencial/tiempo_torch_cpu:.2f}x")

Tiempo de ejecución con PyTorch (GPU): 1.9788742065429688e-05 segundos
Speedup (PyTorch GPU): 72409.08x
